In [106]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from matplotlib.pyplot import savefig, figure
from itertools import combinations

## Стили

In [107]:
sns.set_style({'font.family':'serif', 'font.serif':'Times New Roman'})
font_size = 30
rc={'axes.labelsize': font_size, 'font.size': font_size, 'legend.fontsize': font_size, 'axes.titlesize': font_size}
plt.rcParams.update(**rc)
media_dir = 'Media/'

## Функции

In [108]:
def show_colonies(type):
    plt.figure(figsize=(20, 10))

    KA = data.loc[data.Type == type, :].Colony_KA_37
    plt.hist(KA, label = 'КА')
    plt.axvline(KA.median(), color='w', linestyle='dashed', linewidth=2)
    
    UriSelect = data.loc[data.Type == type, :].Colony_UriSelect_37
    plt.hist(UriSelect, label = 'UriSelect')
    plt.axvline(UriSelect.median(), color='r', linestyle='dashed', linewidth=2)
    
    plt.legend(loc="upper left")
    plt.savefig(media_dir + 'colonies_' + str(type))
    plt.close()

for type in types:
    show_colonies(type)

In [109]:
def compare_colony_medians(type, group = None):
    group = group if group else range(0, 1000)
    KA = data.loc[(data.Type == type) & (data.Colony_KA_37.isin(group)), :].Colony_KA_37
    UriSelect = data.loc[(data.Type == type) & (data.Colony_UriSelect_37.isin(group)), :].Colony_UriSelect_37
    U = scipy.stats.mannwhitneyu(KA, UriSelect)
    inference = 'Значимо отличаются' if U[1] < 0.05 else 'Значимо НЕ отличаются'
    group_desc = '' if group[-1] == 999 else 'Группа ' + str(group[0])
    return (type, group_desc, U, inference)

In [110]:
def compare_colony_group_medians(type):
    groups = [range(0, 50), range(50, 70), range(70, 1000)]
    ans = []
    for group in groups:
        ans.append(compare_colony_medians(type, group))
    return ans

In [111]:
def compare_37_degrees(type):
    d = data.loc[data.Type == type]
    kw = scipy.stats.kruskal(d.Days_KA_37,
                              d.Days_UriSelect_37,
                              d.Days_LY_37)
    inference = 'Значимо отличаются' if kw[1] < 0.05 else 'Значимо НЕ отличаются'

    return (type, kw, inference)

In [112]:
def compare_uriselect_28_37(type):
    d = data.loc[data.Type == type]
    U = scipy.stats.mannwhitneyu(d.Days_UriSelect_28, d.Days_UriSelect_37)
    inference = 'Значимо отличаются' if U[1] < 0.05 else 'Значимо НЕ отличаются'
    return (type, U, inference)

In [113]:
def group_median(type):
    groups = [range(0, 50), range(50, 70), range(70, 1000)]
    KA = []
    UriSelect = []
    for group in groups:
        KA_ = data.loc[(data.Type == type) & (data.Colony_KA_37.isin(group)), :].Colony_KA_37
        UriSelect_ = data.loc[(data.Type == type) & (data.Colony_UriSelect_37.isin(group)), :].Colony_UriSelect_37
        KA.append(np.median(KA_))
        UriSelect.append(np.median(UriSelect_))
    return (KA, UriSelect)

## Данные

In [114]:
data = pd.read_excel('Days_Temperature.xlsx')
types = data.Type.unique()
data.Type = data.Type.str.strip()
data.head()

,Colony_KA_37,Colony_UriSelect_37,Days_KA_37,Days_UriSelect_37,Days_LY_37,Days_UriSelect_28,Type
0,70.0,100.0,21.0,17.0,15,16.0,avium
1,88.0,95.0,22.0,14.0,12,14.0,avium
2,81.0,94.0,18.0,12.0,13,14.0,avium
3,90.0,98.0,12.0,11.0,10,9.0,avium
4,87.0,96.0,10.0,6.0,8,7.0,avium


## Рассчитать достоверны ли различия между количеством колоний на КА и UriSelect

In [115]:
for typ in types:
    show_colonies(typ)
    print(compare_colony_medians(typ))

('avium', '', MannwhitneyuResult(statistic=825.0, pvalue=2.910122447375852e-13), 'Значимо отличаются')
('kansasii', '', MannwhitneyuResult(statistic=514.5, pvalue=0.0012624058261079896), 'Значимо отличаются')
('gordonae', '', MannwhitneyuResult(statistic=307.0, pvalue=3.978022962094956e-14), 'Значимо отличаются')
('fortuitum', '', MannwhitneyuResult(statistic=330.0, pvalue=0.00043614432224726914), 'Значимо отличаются')
('peregrinum', '', MannwhitneyuResult(statistic=49.0, pvalue=0.09457449454423805), 'Значимо НЕ отличаются')
('abscessus', '', MannwhitneyuResult(statistic=21.5, pvalue=0.001873271824814753), 'Значимо отличаются')
('septicum', '', MannwhitneyuResult(statistic=8.0, pvalue=0.000812577018631045), 'Значимо отличаются')


## Расчитать число культур с количеством колоний 50 и менее, 50-69, более 70 и сравнить достоверны ли по ним различия для двух сред

In [116]:
for typ in types:
    for result in compare_colony_group_medians(typ):
        print(result)
    print('')

('avium', 'Группа 0', MannwhitneyuResult(statistic=0.0, pvalue=0.0), 'Значимо отличаются')
('avium', 'Группа 50', MannwhitneyuResult(statistic=3.0, pvalue=0.0953839693435643), 'Значимо НЕ отличаются')
('avium', '', MannwhitneyuResult(statistic=706.0, pvalue=7.162140778958471e-11), 'Значимо отличаются')

('kansasii', 'Группа 0', MannwhitneyuResult(statistic=0.0, pvalue=0.0), 'Значимо отличаются')
('kansasii', 'Группа 50', MannwhitneyuResult(statistic=9.5, pvalue=0.30422879204362396), 'Значимо НЕ отличаются')
('kansasii', '', MannwhitneyuResult(statistic=449.0, pvalue=0.10974163013428062), 'Значимо НЕ отличаются')

('gordonae', 'Группа 0', MannwhitneyuResult(statistic=0.0, pvalue=0.0), 'Значимо отличаются')
('gordonae', 'Группа 50', MannwhitneyuResult(statistic=0.0, pvalue=0.0), 'Значимо отличаются')
('gordonae', '', MannwhitneyuResult(statistic=307.0, pvalue=8.28814506420473e-13), 'Значимо отличаются')

('fortuitum', 'Группа 0', MannwhitneyuResult(statistic=0.0, pvalue=0.0), 'Значимо от

## Рассчитать достоверны ли различия между временем появления роста на 3 средах при 37 градусах

In [117]:
for typ in types:
    print(compare_37_degrees(typ))

('avium', KruskalResult(statistic=nan, pvalue=nan), 'Значимо НЕ отличаются')
('kansasii', KruskalResult(statistic=23.70078264490382, pvalue=7.135763375837727e-06), 'Значимо отличаются')
('gordonae', KruskalResult(statistic=nan, pvalue=nan), 'Значимо НЕ отличаются')
('fortuitum', KruskalResult(statistic=0.9106383134243874, pvalue=0.634245511930037), 'Значимо НЕ отличаются')
('peregrinum', KruskalResult(statistic=0.5478946550652627, pvalue=0.7603721248820525), 'Значимо НЕ отличаются')
('abscessus', KruskalResult(statistic=0.29111479028697024, pvalue=0.8645402682371968), 'Значимо НЕ отличаются')
('septicum', KruskalResult(statistic=1.3121449275362234, pvalue=0.5188852784140833), 'Значимо НЕ отличаются')


## Рассчитать достоверно ли различие между временем появления роста на среде UriSelect при 37 и 28 градусах

In [118]:
for typ in types:
    print(compare_uriselect_28_37(typ))

('avium', MannwhitneyuResult(statistic=3413.0, pvalue=0.26727219600986724), 'Значимо НЕ отличаются')
('kansasii', MannwhitneyuResult(statistic=825.0, pvalue=0.44383300814913973), 'Значимо НЕ отличаются')
('gordonae', MannwhitneyuResult(statistic=1374.0, pvalue=0.01168672984715223), 'Значимо отличаются')
('fortuitum', MannwhitneyuResult(statistic=477.0, pvalue=0.027893353550145796), 'Значимо отличаются')
('peregrinum', MannwhitneyuResult(statistic=36.0, pvalue=0.008238976051867271), 'Значимо отличаются')
('abscessus', MannwhitneyuResult(statistic=42.0, pvalue=0.021208748481981756), 'Значимо отличаются')
('septicum', MannwhitneyuResult(statistic=50.0, pvalue=0.48103366025522115), 'Значимо НЕ отличаются')


## Медианы для групп

In [119]:
for typ in types:
    print(typ, group_median(typ))
    print('')

avium ([46.5, 56.0, 83.0], [nan, 64.5, 94.0])

kansasii ([nan, 59.0, 83.5], [nan, 62.0, 85.0])

gordonae ([nan, 66.5, 81.0], [nan, nan, 92.0])

fortuitum ([nan, nan, 90.0], [nan, nan, 94.0])

peregrinum ([nan, nan, 91.0], [nan, nan, 94.0])

abscessus ([nan, nan, 89.5], [nan, nan, 94.0])

septicum ([nan, nan, 91.0], [nan, nan, 95.5])

